In [3]:
import argparse
import os
import random

import numpy as np
import torch
import torch.backends.cudnn as cudnn

import lavis.tasks as tasks
from lavis.common.config import Config
from lavis.common.dist_utils import get_rank, init_distributed_mode, is_main_process
from lavis.common.logger import setup_logger
from lavis.common.optims import (
    LinearWarmupCosineLRScheduler,
    LinearWarmupStepLRScheduler,
)
from lavis.common.registry import registry
from lavis.common.utils import now

from lavis.datasets.builders import *
from lavis.models import *
from lavis.processors import *
from lavis.runners import *
from lavis.tasks import *



/opt/anaconda3/envs/lavis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/lavis/lib/python3.9/site-packages/timm/models/hub.py:4: FutureWarning: Importing from timm.models.hub is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/opt/anaconda3/envs/lavis/lib/python3.9/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/opt/anaconda3/envs/lavis/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  

In [4]:
import sys
import argparse

# Remove Jupyter's auto-generated arguments
sys.argv = [sys.argv[0]]

parser = argparse.ArgumentParser(description="Training")
# factvqa_newsclip_ft

parser.add_argument("--cfg-path", default = "lavis/projects/instructblip2/train/ooc_ft.yaml", help="path to configuration file.")
parser.add_argument(
    "--options",
    nargs="+",
    help="override some settings in the used config, the key-value pair "
    "in xxx=yyy format will be merged into config file (deprecate), "
    "change to --cfg-options instead.",
)

parser.add_argument(
    "--wname", type=str, default=None, help="wandb name",
)
parser.add_argument(
    "--use_lora", type=bool, default=False, help="whether use lora to train LLM",
)

args = parser.parse_args()
# Print the `args` object in Jupyter Notebook
print("Arguments as Namespace:", args)
print("Arguments as Dictionary:", vars(args))



Arguments as Namespace: Namespace(cfg_path='lavis/projects/instructblip2/train/ooc_ft.yaml', options=None, wname=None, use_lora=False)
Arguments as Dictionary: {'cfg_path': 'lavis/projects/instructblip2/train/ooc_ft.yaml', 'options': None, 'wname': None, 'use_lora': False}


In [5]:
cfg = Config(args)
task = tasks.setup_task(cfg)
run_cfg = cfg.run_cfg
datasets = task.build_datasets(cfg)
# model = task.build_model(cfg)

****************
/Users/namle/DATN/Sniffer/datasets/ooc.json


In [4]:
model_config=cfg.model_cfg

In [5]:
model_cls = registry.get_model_class(model_config.arch)

In [6]:
model_cls

lavis.models.blip2_models.blip2_vicuna_instruct.Blip2VicunaInstruct

In [8]:
model_config

{'arch': 'blip2_vicuna_instruct', 'load_finetuned': False, 'load_pretrained': True, 'pretrained': 'llm-ckpt/instruct_blip_vicuna7b_trimmed.pth', 'finetuned': '', 'image_size': 224, 'drop_path_rate': 0, 'use_grad_checkpoint': True, 'vit_precision': 'fp16', 'freeze_vit': True, 'num_query_token': 32, 'llm_model': 'llm-ckpt/vicuna-7b-v1.1', 'prompt': '', 'model_type': 'vicuna7b', 'use_lora': False, 'max_txt_len': 550}

In [7]:
model = model_cls.from_config(model_config)

/opt/anaconda3/envs/lavis/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: llm-ckpt/vicuna-7b-v1.1 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [6]:
task